In [1]:
import tensorflow as tf
from src.dataset import *

In [2]:
train_data, val_data = download_dataset('ted_hrlr_translate/pt_to_en')

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/124 [00:01<?, ? MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Size...:   1%|          | 1/124 [00:04<08:34,  4.19s/ MiB]

Extraction completed...: 0 file [00:04, ? file/s]
Dl Size...:   2%|▏         | 2/124 [00:05<07:00,  3.45s/ MiB]

Extraction completed...: 0 file [00:05, ? file/s]
Dl Size...:   2%|▏         | 3/124 [00:07<05:37,  2.79s/ MiB]

Extraction completed...: 0 file [00:07, ? file/s]
Dl Size...:   3%|▎         | 4/124 [00:08<04:34,  2.29s/ MiB]

Extraction completed...: 0 file [00:08, ? file/s]
Dl Size...:   4%|▍         | 5/124 [00:09<03:48,  1.92s/ MiB]

Extraction completed...: 0 file [00:09, ? file/s]
Dl Size...:   5%|▍         | 6/124 [00:10<03:15,  1.65s/ MiB]

Extraction completed...: 0 file [00:10, ? file/s]
Dl Size...:   6%|▌         | 7/124 [00:11<02:51,

 49%|████▊     | 25174/51785 [00:00<00:00, 90520.65 examples/s]

Shuffling and writing examples to /Users/soumikrakshit/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteQTSGBZ/ted_hrlr_translate-train.tfrecord


355 examples [00:00, 3549.54 examples/s]                        

Shuffling and writing examples to /Users/soumikrakshit/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteQTSGBZ/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /Users/soumikrakshit/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteQTSGBZ/ted_hrlr_translate-test.tfrecord
Dataset ted_hrlr_translate downloaded and prepared to /Users/soumikrakshit/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [3]:
source_tokenizer, target_tokenizer = get_tokenizers(train_data)

In [5]:
sample_string = 'Downy feathers kiss your face and flutter everywhere'
tokenized_string = target_tokenizer.encode(sample_string)
print('Sample string in English: {}'.format(sample_string))
print('Sample string tokenized: {}'.format(tokenized_string))

Sample string in English: Downy feathers kiss your face and flutter everywhere
Sample string tokenized: [7899, 1383, 113, 7206, 388, 9, 1519, 1117, 76, 646, 4, 1220, 3165, 7863, 1392]


In [6]:
sample_string = 'Penas felpudas beijam seu rosto e flutuam por toda parte'
tokenized_string = source_tokenizer.encode(sample_string)
print('Sample string in Portuguese: {}'.format(sample_string))
print('Sample string tokenized: {}'.format(tokenized_string))

Sample string in Portuguese: Penas felpudas beijam seu rosto e flutuam por toda parte
Sample string tokenized: [8038, 2641, 17, 5242, 1174, 43, 3999, 1625, 7990, 109, 5376, 6, 2839, 50, 23, 166, 962]
